In [ ]:
# Financial Analyst AI Agent
# A tool-use agentic system that provides financial analysis and stock market insights

import os
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, RemoveMessage
from langchain_core.messages import trim_messages

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Import financial analysis tools and libraries
import json
import requests
from datetime import datetime, timedelta
from tqdm import tqdm
from bs4 import BeautifulSoup
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool

# Define the system prompt for the financial analyst agent
AGENT_SYS_PROMPT = """Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks or advice based on general market data.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks or general advice on useful stocks based on general market data and trends.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Starting Flow:
Input validation: Determine if the input is asking about a specific company
or stock ticker (Flow 2). If not, check if they are asking for general advice on potentially useful stocks
based on current market data (Flow 1). Otherwise, respond in a friendly, positive, professional tone
that you don't have information to answer as you can only provide financial advice based on market data.
For each of the flows related to valid questions use the following instructions:

Flow 1:
A. Market Analysis: If the query is valid and the user wants to get general advice on the market
or stocks worth looking into for investing, leverage the general market data tool to get relevant data.
In case you need more information then you can also use web search.

Flow 2:
A. Symbol extraction. If the query is valid and is related to a specific company or companies,
extract the company name or ticker symbol from the question.
If a company name is given, look up the ticker symbol using a tool.
If the ticker symbol is not found based on the company, try to
correct the spelling and try again, like changing "microsfot" to "microsoft",
or broadening the search, like changing "southwest airlines" to a shorter variation
like "southwest" and increasing "limit" to 10 or more. If the company or ticker is
still unclear based on the question or conversation so far, and the results of the
symbol lookup, then ask the user to clarify which company or ticker.

B. Information retrieval. Determine what data the user is seeking on the symbol
identified. Use the appropriate tools to fetch the requested information. Only use
data obtained from the tools. You may use multiple tools in a sequence. For instance,
first determine the company's symbol, then retrieve price data using the symbol
and fundamental indicator data etc. For specific queries only retrieve data using the most relevant tool.
If detailed analysis is needed, you can call multiple tools to retrieve data first.
In case you still need more information then you can also use web search.

Response Generation Flow:
Compose Response: Analyze the retrieved data carefully and provide a comprehensive answer to the user in a clear and concise format,
in a friendly professional tone, emphasizing the data retrieved.
If the user asks for recommendations you can give some recommendations
but emphasize the user to do their own research before investing.
When generating the final response in markdown,
if there are special characters in the text, such as the dollar symbol,
ensure they are escaped properly for correct rendering e.g $25.5 should become \\$25.5

Example Interaction:
User asks: "What is the PE ratio for Eli Lilly?"
Chatbot recognizes 'Eli Lilly' as a company name.
Chatbot uses symbol lookup to find the ticker for Eli Lilly, returning LLY.
Chatbot retrieves the PE ratio using the proper function with symbol LLY.
Chatbot responds: "The PE ratio for Eli Lilly (symbol: LLY) as of May 12, 2024 is 30."

Check carefully and only call the tools which are specifically named below.
Only use data obtained from these tools.
"""

# Define the state of the graph
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Define financial tools
tavily_search = TavilySearchAPIWrapper()

@tool
def search_web(query: str, num_results=8) -> list:
    """Search the web for a query. Userful for general information or general news"""
    results = tavily_search.raw_results(query=query,
                                      max_results=num_results,
                                      search_depth='advanced',
                                      include_answer=False,
                                      include_raw_content=True)
    return results

@tool
def get_stock_ticker_symbol(stock_name: str) -> str:
    """Get the symbol, name and CIK for any publicly traded company"""
    # Use OpenBB to search for stock ticker symbol and company details by name.
    # The provider "sec" fetches data from the U.S. Securities and Exchange Commission (SEC).
    res = obb.equity.search(stock_name, provider="sec")

    # Convert the result to a DataFrame and format it as markdown for readability.
    stock_ticker_details = res.to_df().to_markdown()

    # Prepare the output with the stock details.
    output = """Here are the details of the company and its stock ticker symbol:\n\n""" + stock_ticker_details
    return output

@tool
def get_stock_price_metrics(stock_ticker: str) -> str:
    """Get historical stock price data, stock price quote and price performance data
       like price changes for a specific stock ticker"""

    # Fetch the latest stock price quote using "cboe" provider.
    res = obb.equity.price.quote(stock_ticker, provider='cboe')
    price_quote = res.to_df().to_markdown()

    # Retrieve stock price performance metrics (e.g., percentage change) using "finviz" provider.
    res = obb.equity.price.performance(symbol=stock_ticker, provider='finviz')
    price_performance = res.to_df().to_markdown()

    # Fetch historical price data for the past year using "yfinance" provider.
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=365)).strftime("%Y-%m-%d")
    res = obb.equity.price.historical(symbol=stock_ticker, start_date=start_date,
                                    interval='1d', provider='yfinance')
    price_historical = res.to_df().to_markdown()

    # Combine the results into a formatted output.
    output = ("""Here are the stock price metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
            "Price Quote Metrics:\n\n" + price_quote +
            "\n\nPrice Performance Metrics:\n\n" + price_performance +
            "\n\nPrice Historical Data:\n\n" + price_historical)
    return output

@tool
def get_stock_fundamental_indicator_metrics(stock_ticker: str) -> str:
    """Get fundamental indicator metrics for a specific stock ticker"""

    # Retrieve fundamental financial ratios (e.g., P/E ratio, ROE) using "fmp" provider.
    res = obb.equity.fundamental.ratios(symbol=stock_ticker, period='annual',
                                      limit=10, provider='fmp')
    fundamental_ratios = res.to_df().to_markdown()

    # Fetch additional fundamental metrics (e.g., EBITDA, revenue growth) using "yfinance" provider.
    res = obb.equity.fundamental.metrics(symbol=stock_ticker, period='annual',
                                      limit=10, provider='yfinance')
    fundamental_metrics = res.to_df().to_markdown()

    # Combine fundamental ratios and metrics into a single output.
    output = ("""Here are the fundamental indicator metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
            "Fundamental Ratios:\n\n" + fundamental_ratios +
            "\n\nFundamental Metrics:\n\n" + fundamental_metrics)
    return output

@tool
def get_stock_news(stock_ticker: str) -> str:
    """Get news article headlines for a specific stock ticker"""

    # Define the date range to fetch news (last 45 days).
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=45)).strftime("%Y-%m-%d")

    # Retrieve news headlines for the stock using "tmx" provider.
    res = obb.news.company(symbol=stock_ticker, start_date=start_date, provider='tmx', limit=50)
    news = res.to_df()

    # Extract relevant columns (symbols and titles) and format as markdown.
    news = news[['symbols', 'title']].to_markdown()

    # Prepare the output with the news headlines.
    output = ("""Here are the recent news headlines for the stock ticker symbol """ + stock_ticker + """: \n\n""" + news)
    return output

@tool
def get_general_market_data() -> str:
    """Get general data and indicators for the whole stock market including,
       most actively traded stocks based on volume, top price gainers and top price losers.
       Useful when you want an overview of the market and what stocks to look at."""

    # Retrieve the most actively traded stocks using "yfinance" provider.
    res = obb.equity.discovery.active(sort='desc', provider='yfinance', limit=15)
    most_active_stocks = res.to_df().to_markdown()

    # Fetch the top price gainers using "yfinance" provider.
    res = obb.equity.discovery.gainers(sort='desc', provider='yfinance', limit=15)
    price_gainers = res.to_df().to_markdown()

    # Retrieve the top price losers using "yfinance" provider.
    res = obb.equity.discovery.losers(sort='desc', provider='yfinance', limit=15)
    price_losers = res.to_df().to_markdown()

    # Combine the market data into a single formatted output.
    output = ("""Here's some detailed information of the stock market which includes most actively traded stocks, gainers and losers:\n\n""" +
            "Most actively traded stocks:\n\n" + most_active_stocks +
            "\n\nTop price gainers:\n\n" + price_gainers +
            "\n\nTop price losers:\n\n" + price_losers)
    return output

# List of tools the agent can use
tools = [
    get_stock_ticker_symbol,
    get_stock_price_metrics,
    get_stock_fundamental_indicator_metrics,
    get_stock_news,
    search_web,
    get_general_market_data
]

# Build the graph for the agentic system
def build_financial_analyst_agent(openai_api_key):
    """Build and return the financial analyst agent"""

    # Initialize the graph builder
    graph_builder = StateGraph(State)

    # Initialize the LLM and bind tools
    llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=openai_api_key)
    llm_with_tools = llm.bind_tools(tools)

    # System message to guide the agent's behavior
    SYS_MSG = SystemMessage(content=AGENT_SYS_PROMPT)

    # Define the chatbot node function
    def chatbot(state: State):
        # Trim messages to avoid exceeding token limits
        messages = trim_messages(
            state["messages"],
            max_tokens=127000,
            strategy="last",  # keep last 127K tokens in messages
            token_counter=ChatOpenAI(model="gpt-4o", api_key=openai_api_key),
            include_system=True,  # keep system message always
            allow_partial=True,  # trim messages to partial content if needed
        )
        # Invoke the LLM with the system message and trimmed conversation history
        return {"messages": [llm_with_tools.invoke([SYS_MSG] + messages)]}

    # Add the chatbot node to the graph
    graph_builder.add_node("chatbot", chatbot)

    # Add a tool node for executing tools
    tool_node = ToolNode(tools=tools)
    graph_builder.add_node("tools", tool_node)

    # Add conditional edges for routing between nodes
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
        ['tools', END]
    )

    # After using a tool, return to the chatbot
    graph_builder.add_edge("tools", "chatbot")

    # Set the chatbot as the entry point
    graph_builder.set_entry_point("chatbot")

    # Compile the graph into a runnable agent
    return graph_builder.compile()

# Function to initialize OpenBB and run the agent
def initialize_agent(openai_api_key, openbb_pat, tavily_api_key):
    """Initialize the financial analyst agent with required API keys"""

    # Set environment variables
    os.environ['OPENAI_API_KEY'] = openai_api_key
    os.environ['TAVILY_API_KEY'] = tavily_api_key

    # Initialize OpenBB
    from openbb import obb
    obb.account.login(pat=openbb_pat)

    # Build and return the financial analyst agent
    return build_financial_analyst_agent(openai_api_key)

# Function to call the agent with a query
def get_financial_analysis(agent, query):
    """Call the agent with a financial query and return the response"""

    # Create a configuration for the agent
    user_config = {"configurable": {"thread_id": "financial_analysis"}}

    # Stream the agent's response
    events = agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        user_config,
        stream_mode="values",
    )

    # Get the final response
    for event in events:
        final_response = event["messages"][-1].content

    return final_response

# Example usage (commented out for import compatibility)
"""
# Set your API keys
OPENAI_KEY = "your_openai_key"
OPENBB_PAT = "your_openbb_pat"
TAVILY_API_KEY = "your_tavily_key"

# Initialize the agent
financial_analyst = initialize_agent(OPENAI_KEY, OPENBB_PAT, TAVILY_API_KEY)

# Get financial analysis
query = "What is the PE ratio of Nvidia compared to Intel? Which one is a better investment?"
analysis = get_financial_analysis(financial_analyst, query)
print(analysis)
"""

if __name__ == "__main__":
    print("Financial Analyst AI Agent")
    print("Please set your API keys and run the agent.")